In [1]:
!pip install annoy
!pip install implicit

In [2]:
import annoy
import numpy as np
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import SVD, evaluate, accuracy
from surprise import dump
from surprise.model_selection import cross_validate, train_test_split
from surprise.model_selection import GridSearchCV 

In [49]:
from surprise import NMF
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import KNNWithZScore
from surprise import SlopeOne
from surprise import SVDpp
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import CoClustering

In [28]:
from tqdm import tqdm_notebook as tqdm

In [70]:
import implicit

In [33]:
df = pd.read_csv('item_item_test.csv')
df.head(30)

,Unnamed: 0,userId,movieId,rating,timestamp
0,90,4,181,0.5,1127946789
1,296,4,1760,0.5,1114050751
2,484,4,3593,1.0,1115782046
3,1631,19,181,2.0,836433987
4,2655,38,4125,1.0,982853654
5,4450,54,810,3.0,868126354
6,5189,56,4255,3.5,1381372980
7,6455,72,90890,3.5,1459980875
8,6506,72,125916,0.5,1459979959
9,7119,75,125916,2.5,1511888779


In [34]:
df.drop(['Unnamed: 0','timestamp'], inplace=True,axis=1)

In [35]:
df.head()

,userId,movieId,rating
0,4,181,0.5
1,4,1760,0.5
2,4,3593,1.0
3,19,181,2.0
4,38,4125,1.0


In [36]:
len(df)

63163

In [61]:
def rec_gridsearch(model,param_grid,cv):
    gs = GridSearchCV(model, param_grid, measures=['rmse', 'mae'], cv=cv, n_jobs=-1)
    gs.fit(data)
    # best RMSE score
    print(gs.best_score['rmse'])

    # combination of parameters that gave the best RMSE score
    print(gs.best_params['rmse'])

## Lets narrow down candidates.    

In [51]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=10, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse') 

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

,test_rmse,fit_time,test_time
Algorithm,,,
KNNBaseline,1.020727,89.183144,7.468731
SVDpp,1.040596,14.320231,0.210154
KNNBasic,1.053583,83.150241,6.483075
KNNWithMeans,1.059923,89.239505,6.966381
NMF,1.060838,6.355041,0.069143
KNNWithZScore,1.062559,93.642434,7.276902
SlopeOne,1.064206,0.392056,0.145317
SVD,1.065767,4.401206,0.075216
CoClustering,1.081905,4.598551,0.051666


## SVD

In [14]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df,reader)

In [40]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.0480  1.0639  1.0572  1.0631  1.0695  1.0577  1.0587  1.0828  1.0779  1.0816  1.0660  0.0110  
MAE (testset)     0.8468  0.8610  0.8527  0.8584  0.8670  0.8529  0.8580  0.8763  0.8746  0.8800  0.8628  0.0107  
Fit time          4.23    4.24    4.17    4.20    4.20    4.22    4.29    4.22    4.29    4.16    4.22    0.04    
Test time         0.06    0.06    0.06    0.06    0.06    0.07    0.06    0.06    0.14    0.06    0.07    0.03    


{'test_rmse': array([1.04799704, 1.06393136, 1.05718242, 1.0631157 , 1.06948358,
        1.0576798 , 1.05870252, 1.08278422, 1.07792866, 1.0815916 ]),
 'test_mae': array([0.84680658, 0.86103314, 0.85271596, 0.85842769, 0.86704902,
        0.85287666, 0.85803869, 0.87626983, 0.87460099, 0.88001262]),
 'fit_time': (4.225666761398315,
  4.244113922119141,
  4.16943097114563,
  4.201759099960327,
  4.200515985488892,
  4.22236704826355,
  4.287075757980347,
  4.219311237335205,
  4.285580635070801,
  4.1620237827301025),
 'test_time': (0.05500006675720215,
  0.05860090255737305,
  0.057804107666015625,
  0.05599403381347656,
  0.05787920951843262,
  0.07165002822875977,
  0.05598306655883789,
  0.0560460090637207,
  0.1419200897216797,
  0.06035280227661133)}

In [42]:
param_grid = {'n_epochs': [10, 20, 50], 'n_factors': [1,5,10],'lr_all': [0.001, 0.005, 0.01,0.05],
              'reg_all': [0.4, 0.6], 'biased': [True, False],}
rec_gridsearch(SVD,param_grid,3)

1.0465191432486414
{'n_epochs': 50, 'n_factors': 10, 'lr_all': 0.01, 'reg_all': 0.4, 'biased': True}


In [53]:
param_grid = {'n_epochs': [50,75,100], 'n_factors': [5,10,20],'lr_all': [0.01,0.05],
              'reg_all': [0.3,0.4,0.5], 'biased': [True, False],}
rec_gridsearch(SVD,param_grid,3)

1.0424537695080438
{'n_epochs': 75, 'n_factors': 20, 'lr_all': 0.01, 'reg_all': 0.3, 'biased': True}


In [64]:
param_grid = {'n_epochs': [75], 'n_factors': [20,30,40],'lr_all': [0.01],
              'reg_all': [0.1,0.2,0.3], 'biased': [True],}
rec_gridsearch(SVD,param_grid,3)

1.0298727259675857
{'n_epochs': 75, 'n_factors': 40, 'lr_all': 0.01, 'reg_all': 0.1, 'biased': True}


In [65]:
param_grid = {'n_epochs': [75], 'n_factors': [40,50,60],'lr_all': [0.01],
              'reg_all': [0.01,0.05,0.1], 'biased': [True],}
rec_gridsearch(SVD,param_grid,3)

1.0282990610139882
{'n_epochs': 75, 'n_factors': 60, 'lr_all': 0.01, 'reg_all': 0.1, 'biased': True}


In [66]:
param_grid = {'n_epochs': [75], 'n_factors': [60,70,80,90],'lr_all': [0.01],
              'reg_all': [0.1], 'biased': [True],}
rec_gridsearch(SVD,param_grid,3)

1.0281911365332073
{'n_epochs': 75, 'n_factors': 90, 'lr_all': 0.01, 'reg_all': 0.1, 'biased': True}


Marginal improvement at this point. Moving on to other models.

## NMF

In [37]:
nmf = NMF()
cross_validate(nmf, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.0633  1.0634  1.0668  1.0630  1.0625  1.0599  1.0577  1.0702  1.0516  1.0759  1.0634  0.0063  
MAE (testset)     0.7956  0.7947  0.7967  0.8000  0.8004  0.7915  0.7877  0.7993  0.7878  0.8044  0.7958  0.0053  
Fit time          7.55    7.07    7.60    7.27    7.57    7.09    7.65    7.90    7.39    7.00    7.41    0.28    
Test time         0.07    0.06    0.06    0.07    0.06    0.05    0.06    0.09    0.08    0.15    0.08    0.03    


{'test_rmse': array([1.06328433, 1.06336099, 1.06677292, 1.06301131, 1.06252209,
        1.05991413, 1.05772675, 1.07020364, 1.05158049, 1.07592665]),
 'test_mae': array([0.79562841, 0.79471426, 0.7966913 , 0.79999204, 0.80043335,
        0.79146986, 0.78771769, 0.79933589, 0.78783922, 0.80444307]),
 'fit_time': (7.5549468994140625,
  7.073349952697754,
  7.60212516784668,
  7.266840934753418,
  7.566035985946655,
  7.087186813354492,
  7.645038843154907,
  7.904263973236084,
  7.385869979858398,
  7.002677917480469),
 'test_time': (0.07328200340270996,
  0.05996417999267578,
  0.06066727638244629,
  0.06553912162780762,
  0.059427738189697266,
  0.054814815521240234,
  0.06008100509643555,
  0.08710718154907227,
  0.0838470458984375,
  0.1524968147277832)}

In [39]:
# Not tuning LR/Reg until biased is determined to be true or not. 
param_grid = {'n_epochs': [10, 20, 50], 'n_factors': [1,5,10], 'biased': [True, False],}
tqdm(rec_gridsearch(NMF,param_grid,9))

1.057280091186513
{'n_epochs': 50, 'n_factors': 1, 'biased': False}


0it [00:00, ?it/s]

### KNN Baseline

In [78]:
knn = KNNBaseline()

In [79]:
cross_validate(knn, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

{'test_rmse': array([1.02117399, 1.01472005, 1.01711235, 1.00567749, 1.01832867,
        1.01538854, 1.02541524, 1.03927229, 1.01674642, 1.02055621]),
 'test_mae': array([0.78972274, 0.79656566, 0.79622076, 0.78623501, 0.79834623,
        0.79404881, 0.80474513, 0.81302319, 0.79518576, 0.79507834]),
 'fit_time': (81.97987985610962,
  85.09711527824402,
  83.33498191833496,
  82.5072431564331,
  81.26806092262268,
  85.20557284355164,
  85.35262894630432,
  82.57301998138428,
  82.91954588890076,
  80.7125871181488),
 'test_time': (6.315085172653198,
  6.4196648597717285,
  7.319612741470337,
  6.577861309051514,
  6.937384843826294,
  6.732741832733154,
  6.29314398765564,
  6.4974470138549805,
  7.580524206161499,
  6.67586612701416)}

In [83]:
param_grid_knn = {'k': [20,30,50,60]}
knnbaseline_gs = GridSearchCV(KNNBaseline, param_grid_knn, measures=['rmse', 'mae'], cv=4, n_jobs=-1)
knnbaseline_gs.fit(data)

In [85]:
knnbaseline_gs.best_params
knnbaseline_gs.best_score

{'rmse': 1.0339278002256562, 'mae': 0.81000248363954}

In [86]:
param_grid_knn = {'k': [30,35,40,45]}
knnbaseline_gs = GridSearchCV(KNNBaseline, param_grid_knn, measures=['rmse', 'mae'], cv=4, n_jobs=-1)
knnbaseline_gs.fit(data)
print(knnbaseline_gs.best_params)
print(knnbaseline_gs.best_score)

{'rmse': {'k': 35}, 'mae': {'k': 30}}
{'rmse': 1.0327849906778375, 'mae': 0.8090094874055599}


### SVDpp

In [54]:
svdpp = SVDpp()
cross_validate(svdpp, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.0417  1.0315  1.0367  1.0386  1.0307  1.0364  1.0249  1.0468  1.0484  1.0428  1.0379  0.0070  
MAE (testset)     0.8153  0.8088  0.8099  0.8155  0.7991  0.8077  0.8022  0.8201  0.8223  0.8163  0.8117  0.0071  
Fit time          13.81   13.53   13.70   14.03   13.13   12.79   12.96   12.73   12.63   12.64   13.19   0.50    
Test time         0.31    0.15    0.18    0.16    0.14    0.15    0.14    0.15    0.14    0.15    0.17    0.05    


{'test_rmse': array([1.04172288, 1.03148326, 1.0367124 , 1.03858046, 1.03072872,
        1.03638924, 1.02494009, 1.04676632, 1.04843384, 1.04284449]),
 'test_mae': array([0.81527772, 0.80879415, 0.80990328, 0.8154974 , 0.79912627,
        0.80766324, 0.80224503, 0.82013032, 0.82229712, 0.81628458]),
 'fit_time': (13.805808782577515,
  13.52705192565918,
  13.697068929672241,
  14.026280879974365,
  13.12869930267334,
  12.78980016708374,
  12.958942174911499,
  12.731549978256226,
  12.634726285934448,
  12.638183832168579),
 'test_time': (0.31453514099121094,
  0.14533495903015137,
  0.18267416954040527,
  0.15613913536071777,
  0.14376091957092285,
  0.1474299430847168,
  0.1381058692932129,
  0.15408015251159668,
  0.1385819911956787,
  0.15295100212097168)}

In [62]:
param_grid_svdpp = {'n_epochs': [50,75,100], 'n_factors': [5,10,20],'lr_all': [0.01,0.05],
              'reg_all': [0.3,0.4,0.5],'random_state':[7]}
rec_gridsearch(SVDpp,param_grid_svdpp,7)

1.031005864411734
{'n_epochs': 75, 'n_factors': 20, 'lr_all': 0.01, 'reg_all': 0.3, 'random_state': 7}


In [67]:
param_grid_svdpp = {'n_epochs': [75], 'n_factors': [20,30,40],'lr_all': [0.01],
              'reg_all': [0.1,0.2,0.3],'random_state':[7]}
rec_gridsearch(SVDpp,param_grid_svdpp,7)

1.013783707348949
{'n_epochs': 75, 'n_factors': 40, 'lr_all': 0.01, 'reg_all': 0.1, 'random_state': 7}


In [68]:
param_grid_svdpp = {'n_epochs': [75], 'n_factors': [40,50,60],'lr_all': [0.01],
              'reg_all': [0.05,0.1],'random_state':[7]}
rec_gridsearch(SVDpp,param_grid_svdpp,7)

1.012264700140572
{'n_epochs': 75, 'n_factors': 50, 'lr_all': 0.01, 'reg_all': 0.1, 'random_state': 7}


### Annoy